## Make wordsearch grid

In [1]:
# following B. Helmig's Crossword Puzzle Generator
# http://bryanhelmig.com/python-crossword-puzzle-generator/

In [2]:
import random, re, time, string
from copy import copy as duplicate

In [3]:
class Crossword(object):
    def __init__(self, cols, rows, empty = '-', maxloops = 2000, available_words=[]):
        self.cols = cols
        self.rows = rows
        self.empty = empty
        self.maxloops = maxloops
        self.available_words = available_words
        self.randomize_word_list()
        self.current_word_list = []
        self.debug = 0
        self.clear_grid()
        self.latin_lower = string.ascii_lowercase.replace('j','').replace('v', '')

    def clear_grid(self): # initialize grid and fill with empty character
        self.grid = []
        for i in range(self.rows):
            ea_row = []
            for j in range(self.cols):
                ea_row.append(self.empty)
            self.grid.append(ea_row)
 
    def randomize_word_list(self): # also resets words and sorts by length
        temp_list = []
        for word in self.available_words:
            if isinstance(word, Word):
                temp_list.append(Word(word.word, word.clue))
            else:
                temp_list.append(Word(word[0], word[1]))
        random.shuffle(temp_list) # randomize word list
        temp_list.sort(key=lambda i: len(i.word), reverse=True) # sort by length
        self.available_words = temp_list
 
    def compute_crossword(self, time_permitted = 1.00, spins=2):
        time_permitted = float(time_permitted)
 
        count = 0
        copy = Crossword(self.cols, self.rows, self.empty, self.maxloops, self.available_words)
 
        start_full = float(time.time())
        while (float(time.time()) - start_full) < time_permitted or count == 0: # only run for x seconds
            self.debug += 1
            copy.current_word_list = []
            copy.clear_grid()
            copy.randomize_word_list()
            x = 0
            while x < spins: # spins; 2 seems to be plenty
                for word in copy.available_words:
                    if word not in copy.current_word_list:
                        copy.fit_and_add(word)
                x += 1
            #print copy.solution()
            #print len(copy.current_word_list), len(self.current_word_list), self.debug
            # buffer the best crossword by comparing placed words
            if len(copy.current_word_list) > len(self.current_word_list):
                self.current_word_list = copy.current_word_list
                self.grid = copy.grid
            count += 1
        return
 
    def suggest_coord(self, word):
        count = 0
        coordlist = []
        glc = -1
        for given_letter in word.word: # cycle through letters in word
            glc += 1
            rowc = 0
            for row in self.grid: # cycle through rows
                rowc += 1
                colc = 0
                for cell in row: # cycle through  letters in rows
                    colc += 1
                    if given_letter == cell: # check match letter in word to letters in row
                        try: # suggest vertical placement 
                            if rowc - glc > 0: # make sure we're not suggesting a starting point off the grid
                                if ((rowc - glc) + word.length) <= self.rows: # make sure word doesn't go off of grid
                                    coordlist.append([colc, rowc - glc, 1, colc + (rowc - glc), 0])
                        except: pass
                        try: # suggest horizontal placement 
                            if colc - glc > 0: # make sure we're not suggesting a starting point off the grid
                                if ((colc - glc) + word.length) <= self.cols: # make sure word doesn't go off of grid
                                    coordlist.append([colc - glc, rowc, 0, rowc + (colc - glc), 0])
                        except: pass
        # example: coordlist[0] = [col, row, vertical, col + row, score]
        #print word.word
        #print coordlist
        new_coordlist = self.sort_coordlist(coordlist, word)
        #print new_coordlist
        return new_coordlist
 
    def sort_coordlist(self, coordlist, word): # give each coordinate a score, then sort
        new_coordlist = []
        for coord in coordlist:
            col, row, vertical = coord[0], coord[1], coord[2]
            coord[4] = self.check_fit_score(col, row, vertical, word) # checking scores
            if coord[4]: # 0 scores are filtered
                new_coordlist.append(coord)
        random.shuffle(new_coordlist) # randomize coord list; why not?
        new_coordlist.sort(key=lambda i: i[4], reverse=True) # put the best scores first
        return new_coordlist
 
    def fit_and_add(self, word): # doesn't really check fit except for the first word; otherwise just adds if score is good
        fit = False
        count = 0
        coordlist = self.suggest_coord(word)
 
        while not fit and count < self.maxloops:
 
            if len(self.current_word_list) == 0: # this is the first word: the seed
                # top left seed of longest word yields best results (maybe override)
                vertical, col, row = random.randrange(0, 2), 1, 1
                ''' 
                # optional center seed method, slower and less keyword placement
                if vertical:
                    col = int(round((self.cols + 1)/2, 0))
                    row = int(round((self.rows + 1)/2, 0)) - int(round((word.length + 1)/2, 0))
                else:
                    col = int(round((self.cols + 1)/2, 0)) - int(round((word.length + 1)/2, 0))
                    row = int(round((self.rows + 1)/2, 0))
                # completely random seed method
                '''
                col = random.randrange(1, self.cols + 1)
                row = random.randrange(1, self.rows + 1)
 
                if self.check_fit_score(col, row, vertical, word): 
                    fit = True
                    self.set_word(col, row, vertical, word, force=True)
            else: # a subsquent words have scores calculated
                try: 
                    col, row, vertical = coordlist[count][0], coordlist[count][1], coordlist[count][2]
                except IndexError: return # no more cordinates, stop trying to fit
 
                if coordlist[count][4]: # already filtered these out, but double check
                    fit = True 
                    self.set_word(col, row, vertical, word, force=True)
 
            count += 1
        return
 
    def check_fit_score(self, col, row, vertical, word):
        '''
        And return score (0 signifies no fit). 1 means a fit, 2+ means a cross.
 
        The more crosses the better.
        '''
        if col < 1 or row < 1:
            return 0
 
        count, score = 1, 1 # give score a standard value of 1, will override with 0 if collisions detected
        for letter in word.word:            
            try:
                active_cell = self.get_cell(col, row)
            except IndexError:
                return 0
 
            if active_cell == self.empty or active_cell == letter:
                pass
            else:
                return 0
 
            if active_cell == letter:
                score += 1
 
            if vertical:
                # check surroundings
                if active_cell != letter: # don't check surroundings if cross point
                    if not self.check_if_cell_clear(col+1, row): # check right cell
                        return 0
 
                    if not self.check_if_cell_clear(col-1, row): # check left cell
                        return 0
 
                if count == 1: # check top cell only on first letter
                    if not self.check_if_cell_clear(col, row-1):
                        return 0
 
                if count == len(word.word): # check bottom cell only on last letter
                    if not self.check_if_cell_clear(col, row+1): 
                        return 0
            else: # else horizontal
                # check surroundings
                if active_cell != letter: # don't check surroundings if cross point
                    if not self.check_if_cell_clear(col, row-1): # check top cell
                        return 0
 
                    if not self.check_if_cell_clear(col, row+1): # check bottom cell
                        return 0
 
                if count == 1: # check left cell only on first letter
                    if not self.check_if_cell_clear(col-1, row):
                        return 0
 
                if count == len(word.word): # check right cell only on last letter
                    if not self.check_if_cell_clear(col+1, row):
                        return 0
 
 
            if vertical: # progress to next letter and position
                row += 1
            else: # else horizontal
                col += 1
 
            count += 1
 
        return score
 
    def set_word(self, col, row, vertical, word, force=False): # also adds word to word list
        if force:
            word.col = col
            word.row = row
            word.vertical = vertical
            self.current_word_list.append(word)
 
            for letter in word.word:
                self.set_cell(col, row, letter)
                if vertical:
                    row += 1
                else:
                    col += 1
        return
 
    def set_cell(self, col, row, value):
        self.grid[row-1][col-1] = value
 
    def get_cell(self, col, row):
        return self.grid[row-1][col-1]
 
    def check_if_cell_clear(self, col, row):
        try:
            cell = self.get_cell(col, row)
            if cell == self.empty: 
                return True
        except IndexError:
            pass
        return False
 
    def solution(self): # return solution grid
        outStr = ""
        for r in range(self.rows):
            for c in self.grid[r]:
                outStr += '%s ' % c
            outStr += '\n'
        return outStr
 
    def word_find(self): # return solution grid
        outStr = ""
        for r in range(self.rows):
            for c in self.grid[r]:
                if c == self.empty:
                    outStr += '%s ' % self.latin_lower[random.randint(0,len(self.latin_lower)-1)]
                else:
                    outStr += '%s ' % c
            outStr += '\n'
        return outStr
 
    def order_number_words(self): # orders words and applies numbering system to them
        self.current_word_list.sort(key=lambda i: (i.col + i.row))
        count, icount = 1, 1
        for word in self.current_word_list:
            word.number = count
            if icount < len(self.current_word_list):
                if word.col == self.current_word_list[icount].col and word.row == self.current_word_list[icount].row:
                    pass
                else:
                    count += 1
            icount += 1
 
    def display(self, order=True): # return (and order/number wordlist) the grid minus the words adding the numbers
        outStr = ""
        if order:
            self.order_number_words()
 
        copy = self
 
        for word in self.current_word_list:
            copy.set_cell(word.col, word.row, word.number)
 
        for r in range(copy.rows):
            for c in copy.grid[r]:
                outStr += '%s ' % c
            outStr += '\n'
 
        outStr = re.sub(r'[a-z]', ' ', outStr)
        return outStr
 
    def word_bank(self): 
        outStr = ''
        temp_list = duplicate(self.current_word_list)
        random.shuffle(temp_list) # randomize word list
        for word in temp_list:
            outStr += '%s\n' % word.word
        return outStr
 
    def legend(self): # must order first
        outStr = ''
        for word in self.current_word_list:
            outStr += '%d. (%d,%d) %s: %s\n' % (word.number, word.col, word.row, word.down_across(), word.clue )
        return outStr
 
class Word(object):
    def __init__(self, word=None, clue=None):
        self.word = re.sub(r'\s', '', word.lower())
        self.clue = clue
        self.length = len(self.word)
        # the below are set when placed on board
        self.row = None
        self.col = None
        self.vertical = None
        self.number = None
 
    def down_across(self): # return down or across
        if self.vertical: 
            return 'down'
        else: 
            return 'across'
 
    def __repr__(self):
        return self.word

## Get wordlist from text

### Get poems of Catullus

In [4]:
from cltk.corpus.latin import latinlibrary

In [5]:
catullus_raw = latinlibrary.raw('catullus.txt')

In [6]:
catullus_start = catullus_raw.find(' I.')
catullus_end = catullus_raw.rfind('Fragmenta')
catullus_string = catullus_raw[catullus_start:catullus_end]

In [7]:
# Remove roman numeral headings; must be before lower & replacer
rn_pattern = r'(M{1,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})|M{0,4}(CM|C?D|D?C{1,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,4})|M{0,4}(CM|CD|D?C{0,3})(XC|X?L|L?X{1,3})(IX|IV|V?I{0,3})|M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|I?V|V?I{1,3}))[a|b]?\.'

In [8]:
import re
x = re.finditer(rn_pattern, catullus_string) 

In [9]:
catullus_starts = [m.start(0) for m in re.finditer(rn_pattern, catullus_string)]

In [10]:
# cf. https://stackoverflow.com/a/10851479
catullus_starts.append(None)
catullus_poems = [catullus_string[catullus_starts[i]:catullus_starts[i+1]] for i in range(len(catullus_starts)-1)]

In [11]:
catullus_titles = [item[:item.find('\n')].strip() for item in catullus_poems]
catullus_poems = [item[item.find('\n'):].strip() for item in catullus_poems]

In [12]:
# Imports for preprocessing

import re # Regex module, useful for pattern matching
import html # Useful for handling entities

# Import/load a CLTK tool for normalizing i/j and u/v in Latin texts
from cltk.stem.latin.j_v import JVReplacer
replacer = JVReplacer()

In [13]:
# Preprocess texts
def preprocess(text):

    # Remove html entities and related html artifacts
    text = html.unescape(text) # Handle html entities
    text = re.sub(r'&nbsp;?', ' ',text) #&nbsp; stripped incorrectly in corpus?
    text = re.sub(r'\x00',' ',text) #Another space problem?
    text = re.sub(r' \xa0 ', '    ', text)
    # Lowercase text
    text = text.lower()

    # Normalize text
    text = replacer.replace(text) #Normalize u/v & i/j
    
    # Remove punctuation with translate
    punctuation ="\"#$%&\'()+,-/:;<=>@[\]^_`{|}~.?!«»—"
    translator = str.maketrans({key: " " for key in punctuation})
    text = text.translate(translator)
    
    # Remove numbers
    translator = str.maketrans({key: " " for key in '0123456789'})
    text = text.translate(translator)
        
    return text.strip()

In [14]:
catullus_preprocess = [preprocess(poem) for poem in catullus_poems]

### Compute tf-idf for Catullus

In [15]:
import math
from textblob import TextBlob as tb

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [16]:
bloblist = [tb(poem) for poem in catullus_preprocess]

In [17]:
scores = []
top_words = []

for i, blob in enumerate(bloblist):
    temp = {word: tfidf(word, blob, bloblist) for word in blob.words}
    scores.append(temp)
    sorted_words = sorted(temp.items(), key=lambda x: x[1], reverse=True)
    top_words.append([word[0] for word in sorted_words[:30] if len(word[0]) > 3][:20])

In [18]:
for i, title in enumerate(catullus_titles[:2]):
    print("Top words in Catullus %s" % title)
    sorted_words = sorted(scores[i].items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:20]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))
    print('\n')

Top words in Catullus I. ad Cornelium
	Word: dono, TF-IDF: 0.07995
	Word: lepidum, TF-IDF: 0.07995
	Word: arida, TF-IDF: 0.07995
	Word: expolitum, TF-IDF: 0.07995
	Word: solebas, TF-IDF: 0.07995
	Word: nugas, TF-IDF: 0.07995
	Word: ausus, TF-IDF: 0.07995
	Word: italorum, TF-IDF: 0.07995
	Word: aeuum, TF-IDF: 0.07995
	Word: tribus, TF-IDF: 0.07995
	Word: explicare, TF-IDF: 0.07995
	Word: cartis, TF-IDF: 0.07995
	Word: laboriosis, TF-IDF: 0.07995
	Word: habe, TF-IDF: 0.07995
	Word: libelli, TF-IDF: 0.07995
	Word: qualecumque, TF-IDF: 0.07995
	Word: patrona, TF-IDF: 0.07995
	Word: maneat, TF-IDF: 0.07995
	Word: perenne, TF-IDF: 0.07995
	Word: saeclo, TF-IDF: 0.07995


Top words in Catullus II. fletus passeris Lesbiae
	Word: ludere, TF-IDF: 0.14988
	Word: appetenti, TF-IDF: 0.08322
	Word: acris, TF-IDF: 0.08322
	Word: morsus, TF-IDF: 0.08322
	Word: nitenti, TF-IDF: 0.08322
	Word: iocari, TF-IDF: 0.08322
	Word: solaciolum, TF-IDF: 0.08322
	Word: doloris, TF-IDF: 0.08322
	Word: acquiescat, T

In [19]:
catullus_top_words = top_words[1]
print(catullus_top_words)

['ludere', 'appetenti', 'acris', 'morsus', 'nitenti', 'iocari', 'solaciolum', 'doloris', 'acquiescat', 'sicut', 'tristis', 'leuare', 'passer', 'tenere', 'digitum', 'incitare', 'desiderio', 'carum', 'credo', 'grauis']


## Putting it together

In [20]:
words = [[word, None] for word in catullus_top_words]
catullus_word_search = Crossword(20, 20, '-', 5000, words)
catullus_word_search.compute_crossword(2)

In [21]:
print(catullus_word_search.word_bank())

ludere
appetenti
passer
grauis
leuare
credo
tenere
nitenti
digitum
solaciolum
morsus
carum
acquiescat
incitare
acris
iocari
tristis
doloris
desiderio
sicut



In [22]:
print(catullus_word_search.word_find())

y q d l i d z t c x n m w d p f r r g k 
g q c i n s n g r a u i s y s i c u t m 
g g q a w m c i e z r l o c g f a g c w 
y d t f y t t o d g m z l u d e r e e t 
s k e f r l d q o l a k a o l p u l u l 
i o n m b p u d d b u d c a f a m f i k 
g o e p r p c d o l o r i s x h p y n x 
q t r p p y k i n y t d o r k b i u c d 
s l e u a r e r s r e t l b d g o s i b 
m d k r s t z t t a c q u i e s c a t o 
a c r i s b h r m p u t m k s y a t a c 
y g d o e y i i t p d u r t i q r z r x 
t f m o r s u s l e z y q h d z i t e l 
f q b g s k m t q t p c t x e i d a l z 
x g k w y n n i t e n t i e r g n m w y 
n a t p q g t s h n k b z d i n p g t g 
o m b n u k t f p t p i g b o i m b w i 
h g k h p h d i g i t u m r z p d i l p 
y e i i m l u s f e l x z b o o a a u e 
a c x w u x y n d f d a i e k s d o y f 



In [23]:
print(catullus_word_search.solution())

- - - - - - - - c - - - - - - - - - - - 
- - - - - - - g r a u i s - s i c u t - 
- - - - - - - - e - - - o - - - a - - - 
- - t - - - - - d - - - l u d e r e - - 
- - e - - - - - o - - - a - - - u - - - 
- - n - - - - - - - - - c - - - m - i - 
- - e - - - - d o l o r i s - - - - n - 
- - r - p - - - - - - - o - - - i - c - 
- l e u a r e - - - - - l - d - o - i - 
- - - - s - - t - a c q u i e s c a t - 
a c r i s - - r - p - - m - s - a - a - 
- - - - e - - i - p - - - - i - r - r - 
- - m o r s u s - e - - - - d - i - e - 
- - - - - - - t - t - - - - e - - - - - 
- - - - - - n i t e n t i - r - - - - - 
- - - - - - - s - n - - - - i - - - - - 
- - - - - - - - - t - - - - o - - - - - 
- - - - - - d i g i t u m - - - - - - - 
- - - - - - - - - - - - - - - - - - - - 
- - - - - - - - - - - - - - - - - - - - 

